In [2]:
import pandas as pd
import numpy as np

data_dir = "data/"
df_train = pd.read_csv(data_dir + 'train.csv')
df_train.head()

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru


In [3]:
def getInterest(_list, arr_index):
    return _list[arr_index]

df_train['POI'] = df_train['POI/street'].str.split('/').apply(getInterest, args=(0,))
df_train['street'] = df_train['POI/street'].str.split('/').apply(getInterest, args=(1,))
df_train.head(20)

,id,raw_address,POI/street,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,,
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,
4,4,jl. orde baru,/jl. orde baru,,jl. orde baru
5,5,"raya samb gede, 299 toko bb kids",toko bb kids/raya samb gede,toko bb kids,raya samb gede
6,6,"kem mel raya, no 4 bojong rawalumbu rt 1 36 ra...",/kem mel raya,,kem mel raya
7,7,tela keuramat kuta alam,/tela,,tela
8,8,gg. i wates magersari,/gg. i,,gg. i
9,9,bunga ncole ix 2,/bunga ncole ix,,bunga ncole ix


In [4]:
def getListOfWords(sentence):
    sentence = sentence.replace('\n', ' ').replace('\t', ' ')
    sentence = sentence.replace('/', ' ').replace('.', ' ').replace(',', ' ')
    words = sentence.split(' ')
    while '' in words:
        words.remove('')
        
    return [word for word in words if len(words) > 0]

In [5]:
def getDifference(row):
    raw = getListOfWords(row['raw_address'])
    poi_street = getListOfWords(row['POI/street'])
    temp = set(raw)
    output = [x for x in poi_street if x not in temp]
    if len(output) > 0:
        return output
    else:
        return np.nan

In [6]:
from time import time

start = time()

df_train['delta_words'] = df_train.apply(getDifference, axis=1)

print("Executed in {} seconds.".format(time() - start))

Executed in 12.103731870651245 seconds.


In [7]:
df_train.dropna()

,id,raw_address,POI/street,POI,street,delta_words
10,10,"cikahuripan sd neg boj 02 klap boj, no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj,"[negeri, bojong]"
11,11,"yaya atohar,",yayasan atohariyah/,yayasan atohariyah,,"[yayasan, atohariyah]"
20,20,"toko bang ajs,",toko bangunan ajs/,toko bangunan ajs,,[bangunan]
40,40,mar tabl metro iringmulyo metro timur,markaz tabligh metro/,markaz tabligh metro,,"[markaz, tabligh]"
44,44,sd neg 12 anggrek,sd negeri 12 anggrek/,sd negeri 12 anggrek,,[negeri]
...,...,...,...,...,...,...
299973,299973,moha toha no 167,/mohammad toha,,mohammad toha,[mohammad]
299974,299974,"islamic training cen pare, brawi, pelem pare",islamic training center pare/brawi,islamic training center pare,brawi,[center]
299983,299983,"la banda minima, cile raya, pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya,[minimarket]
299986,299986,"ginzi cafe, siliw,",ginzi cafe/siliwangi,ginzi cafe,siliwangi,[siliwangi]


# N-gram analysis

In [8]:
# natural language processing: n-gram ranking
import re
import unicodedata
import nltk

In [9]:
def basic_clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  wnl = nltk.stem.WordNetLemmatizer()
  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return [wnl.lemmatize(word) for word in words]

In [10]:
words_raw = basic_clean(''.join(str(df_train['raw_address'].tolist())))
words_poi = basic_clean(''.join(str(df_train['POI/street'].tolist())))

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
(pd.Series(nltk.ngrams(words_raw, 2)).value_counts())[:20]

In [ ]:
(pd.Series(nltk.ngrams(words_poi, 1)).value_counts())[:20]